In [1]:
import pandas as pd

# ────────────────────────────────────────────────
# 1. 지역 매핑 함수
# ────────────────────────────────────────────────
def extract_region(address):
    region_keywords = {
        '서울시': '서울', '부산시': '부산', '강원도': '강원',
        '경기도': '경기', '경남': '경남', '경북': '경북',
        '광주시': '광주', '대구시': '대구', '대전시': '대전',
        '세종시': '세종', '인천시': '인천', '전남': '전남',
        '전북': '전북', '제주도': '제주', '충남': '충남',
        '충북': '충북'
    }
    for region, keyword in region_keywords.items():
        if pd.notnull(address) and keyword in address:
            return region
    return "기타"

# ────────────────────────────────────────────────
# 2. 연도별 재무 파일 처리 (지역, 업력, PBR 포함)
# ────────────────────────────────────────────────
def preprocess_financial(file_path):
    df = pd.read_excel(file_path)
    df['회계년도'] = df['회계년도'].astype(str).str[:4].astype(int)
    df['발행주식수(주)'] = df['발행주식수(주)'].astype(str)

    # 총발행주식수
    share_sum = df.groupby(['회사명', '회계년도'])['발행주식수(주)'].apply(
        lambda x: x.drop_duplicates().astype(float).sum()
    ).reset_index(name='총발행주식수')

    # 대표 row
    df_first = df.drop_duplicates(['회사명', '회계년도']).copy()

    # 병합
    df_merged = pd.merge(df_first, share_sum, on=['회사명', '회계년도'], how='left')

    # 시가총액
    df_merged['총시가총액'] = df_merged['총발행주식수'] * df_merged['산술평균(원)']

    # 지역
    df_merged['지역'] = df_merged['[공통]본사 주소'].apply(extract_region)

    # 업력
    df_merged['설립연도'] = df_merged['[공통]설립일'].astype(str).str[:4].astype(int)
    df_merged['업력'] = df_merged['회계년도'] - df_merged['설립연도']

    # PBR 평균
    df_merged['PBR'] = df_merged[['[공통]PBR(최저)(IFRS)', '[공통]PBR(최고)(IFRS)']].mean(axis=1)

    return df_merged

# ────────────────────────────────────────────────
# 3. 배당정보 불러오기
# ────────────────────────────────────────────────
def load_dividends(file_path):
    df = pd.read_excel(file_path)
    df['사업연도'] = df['사업연도'].astype(str).str[:4].astype(int)
    df['거래소코드'] = df['종목코드'].astype(str).str.zfill(6)
    df['총배당금액'] = df['총배당금액'].replace("-", 0)
    df['총배당금액'] = df['총배당금액'].astype(str).str.replace(",", "").astype(float)
    return df[['거래소코드', '사업연도', '총배당금액']]

# ────────────────────────────────────────────────
# 4. TSR 계산
# ────────────────────────────────────────────────
def calc_tsr(df_prev, df_curr, dividends, year):
    df_curr = df_curr.copy()
    df_prev = df_prev.copy()
    df_curr['거래소코드'] = df_curr['거래소코드'].astype(str).str.zfill(6)
    df_prev['거래소코드'] = df_prev['거래소코드'].astype(str).str.zfill(6)

    df = pd.merge(df_curr, df_prev[['회사명', '총시가총액']], on='회사명', how='left', suffixes=('', '_전년도'))
    div = dividends[dividends['사업연도'] == year]
    df = pd.merge(df, div, on='거래소코드', how='left')
    df['총배당금액'] = df['총배당금액'].fillna(0)

    df['TSR'] = ((df['총시가총액'] - df['총시가총액_전년도'] + df['총배당금액']) / df['총시가총액_전년도']) * 100
    return df

# ────────────────────────────────────────────────
# 5. 실행
# ────────────────────────────────────────────────

# 코스피
fin_22_kospi = preprocess_financial("2022 코스피.xlsx")
fin_23_kospi = preprocess_financial("2023 코스피.xlsx")
fin_24_kospi = preprocess_financial("2024 코스피.xlsx")
div_kospi = load_dividends("코스피 배당정보.xlsx")

tsr_2023_kospi = calc_tsr(fin_22_kospi, fin_23_kospi, div_kospi, 2023)
tsr_2024_kospi = calc_tsr(fin_23_kospi, fin_24_kospi, div_kospi, 2024)

tsr_2023_kospi.to_excel("TSR_2023_코스피.xlsx", index=False)
tsr_2024_kospi.to_excel("TSR_2024_코스피.xlsx", index=False)

# 코스닥
fin_22_kosdaq = preprocess_financial("2022 코스닥.xlsx")
fin_23_kosdaq = preprocess_financial("2023 코스닥.xlsx")
fin_24_kosdaq = preprocess_financial("2024 코스닥.xlsx")
div_kosdaq = load_dividends("코스닥 배당정보.xlsx")

tsr_2023_kosdaq = calc_tsr(fin_22_kosdaq, fin_23_kosdaq, div_kosdaq, 2023)
tsr_2024_kosdaq = calc_tsr(fin_23_kosdaq, fin_24_kosdaq, div_kosdaq, 2024)

tsr_2023_kosdaq.to_excel("TSR_2023_코스닥.xlsx", index=False)
tsr_2024_kosdaq.to_excel("TSR_2024_코스닥.xlsx", index=False)


In [2]:
import pandas as pd
from pathlib import Path

# ───────────────────────────────
# 1. 공시 참여 기업 종목코드 로드
# ───────────────────────────────
public_df = pd.read_excel("공시현황.xlsx", engine="openpyxl")
public_codes = set(public_df["종목코드"].astype(str).str.strip().str.zfill(6))

# ───────────────────────────────
# 2. 지수 편입 종목코드 로드
# ───────────────────────────────
index_df = pd.read_excel("Kodex_코리아밸류업.xlsx", dtype={"종목코드": str})
index_codes = set(index_df["종목코드"].astype(str).str.strip().str.zfill(6))

# ───────────────────────────────
# 3. TSR 파일 리스트
# ───────────────────────────────
tsr_files = [
    "TSR_2023_코스피.xlsx",
    "TSR_2023_코스닥.xlsx",
    "TSR_2024_코스피.xlsx",
    "TSR_2024_코스닥.xlsx",
]

# ───────────────────────────────
# 4. 각 파일 처리
# ───────────────────────────────
for file in tsr_files:
    df = pd.read_excel(file)
    
    if "거래소코드" not in df.columns:
        print(f"[오류] {file}: '거래소코드' 컬럼 없음 → 건너뜀")
        continue

    # 코드 포맷 정리
    df["거래소코드"] = df["거래소코드"].astype(str).str.strip().str.zfill(6)

    # 기존 더미 제거
    df = df.drop(columns=[col for col in df.columns if "밸류업" in col], errors="ignore")

    # 새 더미 추가
    df["밸류업 참여 여부"] = df["거래소코드"].apply(lambda x: 1 if x in public_codes else 0)
    df["밸류업 지수 참여 여부"] = df["거래소코드"].apply(lambda x: 1 if x in index_codes else 0)

    # 새 파일로 저장
    outname = file.replace(".xlsx", "_처치추가.xlsx")
    df.to_excel(outname, index=False)
    print(f"[완료] {outname} 생성 완료 (행 {len(df):,}개)")


[완료] TSR_2023_코스피_처치추가.xlsx 생성 완료 (행 804개)
[완료] TSR_2023_코스닥_처치추가.xlsx 생성 완료 (행 1,746개)
[완료] TSR_2024_코스피_처치추가.xlsx 생성 완료 (행 808개)
[완료] TSR_2024_코스닥_처치추가.xlsx 생성 완료 (행 1,776개)


In [3]:
import pandas as pd
from pathlib import Path

# 연도 추출을 위해 파일 이름 패턴 활용
files = [
    "TSR_2023_코스피_처치추가.xlsx",
    "TSR_2023_코스닥_처치추가.xlsx",
    "TSR_2024_코스피_처치추가.xlsx",
    "TSR_2024_코스닥_처치추가.xlsx",
]

for file in files:
    df = pd.read_excel(file)
    
    # 연도 추출
    year = int(file.split("_")[1])

    # After 변수 생성
    df["After"] = 1 if year >= 2024 else 0

    # 밸류업 참여 더미는 2024부터만 인정 (이전 연도는 0으로 강제)
    if year < 2024:
        df["밸류업 참여 여부"] = 0
        df["밸류업 지수 참여 여부"] = 0

    # 저장
    output_file = file.replace(".xlsx", "_After포함.xlsx")
    df.to_excel(output_file, index=False)
    print(f"[완료] {output_file} 생성")


[완료] TSR_2023_코스피_처치추가_After포함.xlsx 생성
[완료] TSR_2023_코스닥_처치추가_After포함.xlsx 생성
[완료] TSR_2024_코스피_처치추가_After포함.xlsx 생성
[완료] TSR_2024_코스닥_처치추가_After포함.xlsx 생성
